# 2021 Age

In [1]:
import pandas as pd
import numpy as np
import cleandata as cd

In [2]:
df = pd.read_csv('../data/Gemeinde_Exekutive - daten.csv')

## Clean Data

In [3]:
df = cd.clean(df)

Jahrgänge nicht zugeordnet: 0
Partei nicht zugeordnet: 9
Keine Jahrgänge: 18


# Alles übers Alter

## Das jüngste Gremium

In [ ]:
df_age = df[df.Jahrgang.notna()].copy()
df_age['Alter'] = pd.to_numeric(df_age['Alter'])

df_age = df_age.groupby('Gemeinde').agg({'Alter': 'mean'})

df_age['Alter'] = round(df_age['Alter'], 1)

df_age.sort_values('Alter', ascending=True).head()

## Das älteste Gremium

In [ ]:
df_age.sort_values('Alter', ascending=False).head()

## Die Küken

In [ ]:
df.sort_values('Alter').head(10)

## Die Silberrücken

In [ ]:
df.sort_values('Alter', ascending=False).head(10)

## Jüngsten Frauen

In [4]:
len(df)

1017

In [ ]:
df[df.Geschlecht == 'w'].sort_values('Alter').head(10)

## Alter pro Partei

In [ ]:
df_a = df[df.Alter.notna() & df.partei_nicht_zugeordnet == False].copy()
df_a['Alter'] = pd.to_numeric(df_a['Alter'])
df_a = df_a.groupby('partei_c').agg({'Alter': 'mean'}).sort_values('Alter', ascending=False)
df_a['Alter'] = round(df_a['Alter'], 1)

df_a

# QGIS export

## Aggregate different vectors

In [ ]:
# Find strongest Party. "Multiple" when no single party is the stronges

df_g = df

df_g = pd.pivot_table(df_g, index='Gemeinde', columns='partei_c', values='Name', aggfunc='count').fillna(0)

def maxparty(row):
    maxi = 0
    maxs = ""
    multiple = False
    for i in df_g.columns:
        if i == 'partei_c': continue
        
        if row[i] == maxi:
            multiple = True
        elif row[i] > maxi:
            maxi = row[i]
            maxs = i
            multiple = False
            
    if multiple:
        return 'multiple'
    else:
        return maxs

df_g['topparty'] = df_g.apply(maxparty, axis=1)

# Add "Has Top Party": single, multi or parteilos
def add_has_topparty(x):
    if x == 'multiple': return 'multiple'
    elif x == 'parteilos': return 'parteilos'
    else: return 'single'

df_g['has_topparty'] = df_g['topparty'].apply(add_has_topparty)

df_export = df_g.reset_index()[['Gemeinde', 'topparty', 'has_topparty']].copy()

# Diese Parteien sind so oft die grösste Partei ihrer Gemeinde
df_g.reset_index().groupby('topparty').agg({'Gemeinde': 'count'}).sort_values('Gemeinde', ascending=False)

In [ ]:
# Calc Partys percentage
df_g = pd.pivot_table(df, index='Gemeinde', columns='partei_c', values='Name', aggfunc='count').fillna(0)
df_g['total'] = df_g.sum(axis=1)

def calc_percentage(row):
    for i in df_g.columns:
        row[i] = round(100 / row['total'] * row[i], 1)
    return row

df_g = df_g.apply(calc_percentage, axis=1)

# Drop total
df_g.drop(columns='total', inplace=True)

df_export = df_export.merge(df_g.reset_index(), on='Gemeinde', how='left')

In [ ]:
# Add m / w

df_g = pd.pivot_table(df, index='Gemeinde', columns='Geschlecht', values='Name', aggfunc='count')

df_g['total'] = df_g.sum(axis=1)
df_g['m'] = round(100 / df_g['total'] * df_g['m'])
df_g['w'] = round(100 / df_g['total'] * df_g['w'])

df_export = df_export.merge(df_g.reset_index()[['Gemeinde', 'm', 'w']], on='Gemeinde', how='left')

In [ ]:
# Add age m / w

df_a = df[df.Alter.notna()].copy()
df_a['Alter'] = pd.to_numeric(df_a['Alter'])
df_g = pd.pivot_table(df_a, index='Gemeinde', columns='Geschlecht', values='Alter', aggfunc='mean')

df_g = df_g.apply(lambda x: round(x))

df_g.rename(columns={'m': 'age_m', 'w': 'age_w'}, inplace=True)

df_export = df_export.merge(df_g.reset_index(), on='Gemeinde', how='left')

In [ ]:
# Agg General

df_a = df[df.Alter.notna()].copy()
df_a['Alter'] = pd.to_numeric(df_a['Alter'])

df_a = df_a.groupby('Gemeinde').agg({'Alter': 'mean'})

df_a['Alter'] = round(df_a['Alter'])

df_export = df_export.merge(df_a.reset_index(), on='Gemeinde', how='left')

df_export = df_export.set_index('Gemeinde')

## Join per GeoPandas and Export

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
gdf = gpd.read_file('../data/Gemeindegrenzen.gpkg', layer='UP_GEMEINDEN_F')

# Join
gdf = gdf.merge(df_export, left_on = 'GEMEINDENAME', right_on='Gemeinde', how='left')

# Export
gdf.to_file('../export/gemeinderaete.gpkg', driver='GPKG')

## Parteilose eher am Rande des Kantons (ländlich)

In [ ]:
gdf.plot(column='parteilos', legend=True, figsize=(10, 10))

## Welche Partei hat wo die Mehrheit?

In [ ]:
fig = plt.figure(figsize = (10, 30))

#fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))

c = 1
for i, party in enumerate(gdf['topparty'].unique()):
    
    if not (type(party) == str): continue
    
    ax1 = fig.add_subplot(4, 2, c)
    gdf['leadingparty'] = gdf.topparty.apply(lambda x: party if x == party else None)
    gdf.plot(ax=ax1, column='leadingparty', missing_kwds={'color': 'lightgrey'}, legend=True)
    c += 1
